In [ ]:
# pip install sympy

In [9]:
import math as m
from sympy import diff, lambdify, symbols

# Реализация алгоритма

In [10]:
def my_f(x1, x2):
    """Возвращает значение функции двух переменных в заданной точке."""
    x_1, x_2 = symbols('x_1 x_2')
    expr = 2 * x_1 ** 2 + 5 * x_2 ** 2 + x_1 * x_2 - 3 * x_1 - x_2
    f = lambdify([x_1, x_2], expr, 'numpy')
    return expr, f(x1, x2)

In [11]:
def grad_f(x1, x2):
    """Возвращает градиент функции двух переменных в заданной точке."""
    f = my_f(x1, x2)[0]
    x_1, x_2 = symbols('x_1 x_2')
    dfdx1, dfdx2 = diff(f, x_1), diff(f, x_2)
    partial_1 = lambdify([x_1, x_2], dfdx1, 'numpy')
    partial_2 = lambdify([x_1, x_2], dfdx2, 'numpy')
    return [partial_1(x1, x2), partial_2(x1, x2)]

In [12]:
def grad_norm(x1, x2):
    """
    Возвращает значение нормы градиента функции двух переменных 
    в заданной точке.
    """
    return sum(map(lambda x: m.pow(x, 2), grad_f(x1, x2)))

In [13]:
def dichotomy_method(x1, x2, a=-10, b=20, epsilon=0.0001):
    """
    Реализует метод дихотомии. Возвращает значение шага (alpha) 
    для метода наискорейшего спуска.
    """
    alpha = symbols('alpha')
    x_1 = x1 - alpha * grad_f(x1, x2)[0]
    x_2 = x2 - alpha * grad_f(x1, x2)[1]
    f = my_f(x_1, x_2)[1]                   # минимизируем: в символьном виде
    f_lambda = lambdify(alpha, f, 'numpy')  # минимизируем: в численном виде
    delta = epsilon / 2
    k = 0

    while True:
        k += 1
        alpha_1 = (a + b - delta) / 2
        alpha_2 = alpha_1 + delta
        f_1 = f_lambda(alpha_1)
        f_2 = f_lambda(alpha_2)
    
        epsilon_k = (b - a) / 2

        if epsilon_k <= epsilon:
            break

        if f_1 <= f_2:
            b = alpha_2
        else:
            a = alpha_1
  
    alpha = (b + a) / 2
    return alpha

In [14]:
# реализация алгоритма
x1_0, x2_0 = 1, 1
f_0 = my_f(x1_0, x1_0)[1]
epsilon = 0.0001
k = 1
print('k = %-3.d x1 = %-10.5f x2 = %-10.5f f(X) = %-10.5f' 
        % (k, x1_0, x2_0, f_0))

while True:
    k += 1
    grad = grad_f(x1_0, x2_0)
    alpha = dichotomy_method(x1_0, x2_0)
  
    x1_1 = x1_0 - alpha * grad[0]
    x2_1 = x2_0 - alpha * grad[1]
    f_1 = my_f(x1_1, x2_1)[1]

    x1_0, x2_0 = x1_1, x2_1
    f_0 = f_1

    print('k = %-3.d x1 = %-10.5f x2 = %-10.5f f(X) = %-10.5f' 
        % (k, x1_0, x2_0, f_0))
  
    if grad_norm(x1_0, x2_0) < epsilon:
        break

k = 1   x1 = 1.00000    x2 = 1.00000    f(X) = 4.00000   
k = 2   x1 = 0.80311    x2 = 0.01554    f(X) = -1.12121  
k = 3   x1 = 0.74375    x2 = 0.02635    f(X) = -1.12820  


# Недостатки реализации алгоритма
1. Для функции `dichotomy_method(x1, x2, a=-10, b=20, epsilon=0.0001)` должен быть реализован отдельно поиск отрезка унимодальности;
2. Метод дихотомии лучше импортировать из другого файла;
3. Сам алгоритм стоит записать в виде функции;
4. Избавиться от двух одинаковых `print()` в алгоритме;
5. Нет визуализации.